In [1]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
import numpy as np
import scipy as sp
import pandas as pd
import statsmodels.api as sm
import statsmodels.formula.api as smf
import statsmodels.stats.api as sms
import sklearn as sk
import matplotlib as mpl
import matplotlib.pylab as plt
from mpl_toolkits.mplot3d import Axes3D
import seaborn as sns
sns.set()
sns.set_style("whitegrid")
sns.set_color_codes()

In [2]:
sales = pd.read_csv('../data/train.csv', parse_dates=['date'])
keys = pd.read_csv('../data/key.csv')
weather = pd.read_csv('../data/weather.csv', parse_dates=['date'])
df_1 = pd.merge(weather, keys)
df_1 = pd.merge(df_1, sales)

dates = df_1['date'].dt
df_1['year'] = dates.year
df_1['month'] = dates.month
df_1['day'] = dates.day

final_sample = pd.read_csv('../data/01. final_sample')
trimmed = final_sample.iloc[:, 1:]

In [3]:
def item_nbr_tendency_finely(store_nbr, year, month_start=-1, month_end=-1, graph=True):
    '''
    input
        1. store_nbr = 스토어 번호
        2. year = 연도
        3. month_start = 시작달
        4. month_start = 끝달
        5. graph = 위의 정보에 대한 item_nbr 그래프 출력여부
    
    output
        1. store_nbr, year, month로 filtering한 item_nbr의 pivot 테이블
    '''
    store = df_1[(df_1['store_nbr'] == store_nbr) &
                 (df_1['year'] == year)]

    if month_start != -1:
        if month_end == -1:
            month_end = month_start + 1
        store = store[(month_start <= store['month']) & (store['month'] < month_end)]

    pivot = store.pivot_table(index='item_nbr',
                              columns='date',
                              values='units',
                              aggfunc=np.sum)

    zero_index = pivot == 0
    pivot = pivot[pivot != 0].dropna(axis=0, how='all')
    pivot[zero_index] = 0

    if graph:
        plt.figure(figsize=(12, 8))
        sns.heatmap(pivot, cmap="YlGnBu", annot=True, fmt='.0f')
        plt.show()

    return pivot

In [4]:
def weather_tendency(store_nbr, year, month_start = -1, month_end = -1):
    '''
    input:
        위와 같음
        
    output:
        위의 정보로 filtering한 train,key,weather DataFrame
    '''
    store = df_1[(df_1['store_nbr'] == store_nbr) &
                 (df_1['year'] == year)]
    
    if month_start!=-1:
        if month_end == -1:
            month_end = month_start + 1
        store = store[(month_start <= store['month']) & (store['month'] < month_end)]
    
    store = store.drop(labels=['item_nbr','units'],axis=1)
#     store = store.iloc[:,:]
    
    store = store.drop_duplicates(keep='first').reset_index(drop=True)
    
    store.index.name='date'
    store.index = store['date']
        
    return store
    

In [5]:
def get_correlation(store_nbr, year, month_start=-1, month_end = -1):
    '''
    input:
        위와 같음
    output:
        missing, tracing data를 제외한
        팔린 item_nbr별 각 weather feature에 대한 pearsonr, pvalue를 담은 dictionary
    '''
    
    '''
    Use-case
        1. 7번 store, 2012년 1월부터 12월까지 item_nbr별 상관관계를 알고 싶다
        get_correlation(7,2012)
        
        2. 7번 store, 2012년 1월부터 3월까지 item_nbr별 상관관계를 알고 싶다
        get_correlation(7,2012,1,3)
        
        3. 7번 store, 2012년 5월만 item_nbr별 상관관계를 알고 싶다
        get_correlation(7,2012,5)
    '''
    correlation = dict()

    units_table = item_nbr_tendency_finely(store_nbr, year, month_start, month_end, graph=False)
    weather_table = weather_tendency(store_nbr, year, month_start, month_end)

    weather = list(weather_table.columns[3:20])
    weather.remove('codesum')

    item_nbr = units_table.index

    inner = dict()

    for feature in weather:
        for units in item_nbr:

            a = weather_table[feature].copy()
            b = units_table.loc[units].copy()

            a = a.apply(lambda x: x.replace(' ', ''))
            a = a.apply(lambda x: x.replace('-', ''))

            missing_index = (a.str.contains('M')) | (a.str.contains('T')) | (
                a.str.contains('-')) | (a.str.contains(' '))

            a[missing_index] = np.nan
            b[missing_index] = np.nan

            a.dropna(axis=0, inplace=True)
            b.dropna(axis=0, inplace=True)

            inner[(units, feature)] = sp.stats.pearsonr(a.astype(float), b)

            correlation[(store_nbr, year, month_start, month_end)] = inner

    return correlation

In [6]:
def show_me_pearson(pearson_dict):
    '''
    input:
        get_correlation의 return (상관관계 dictionary)
    output:
        dictionary를 직관적으로 출력함
    '''
    for key,val in get_correlation(7,2012).items():
        print(key)
        for each,each_val in val.items():
            print('\t',each,each_val)

In [7]:
store_nbrs = range(7,8)
years=range(2012,2013)
months = range(1,13)

total_pearsonr = dict()

for store_nbr in store_nbrs:
    for year in years:
        for month in months:
#             print(year,month)
            pearsonr = get_correlation(store_nbr,year,month)
            total_pearsonr.update(pearsonr)

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/scipy/stats/stats.py:3003: RuntimeWarning: invalid value encountered in double_scalars
  r = r_num / r_den


In [8]:
show_me_pearson(get_correlation(7,2012))

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/scipy/stats/stats.py:3003: RuntimeWarning: invalid value encountered in double_scalars
  r = r_num / r_den


(7, 2012, -1, -1)
	 (5, 'tmin') (-0.077513591302145163, 0.13994389337994312)
	 (7, 'tmin') (0.0044011420437656363, 0.9333108321020509)
	 (23, 'tmin') (-0.024866829855208324, 0.63630578218369893)
	 (93, 'tmin') (0.1068799116642834, 0.041553327207246221)
	 (5, 'tavg') (-0.12137801589834839, 0.021070493897311492)
	 (7, 'tavg') (0.0035765695740195644, 0.94600903602056807)
	 (23, 'tavg') (-0.045655442190484015, 0.38709585303678296)
	 (93, 'tavg') (0.052238343715544977, 0.32229035017634811)
	 (5, 'depart') (0.043084346509762353, 0.41441858383189079)
	 (7, 'depart') (-0.056158162413488623, 0.28726322409877886)
	 (23, 'depart') (0.076038079748448686, 0.14935849890868383)
	 (93, 'depart') (0.043086912241579699, 0.41439076406081499)
	 (5, 'dewpoint') (-0.075682965143137187, 0.14901153042300766)
	 (7, 'dewpoint') (0.019924767342382309, 0.70439521919641135)
	 (23, 'dewpoint') (-0.071773870891758199, 0.17121828629513086)
	 (93, 'dewpoint') (0.1459619825857725, 0.0052058353454991213)
	 (5, 'wetbulb'

In [9]:
def get_correlation(store_nbr, year, month = -1):
    correlation = dict()

    units_table = item_nbr_tendency_finely(store_nbr, year, month, graph=False)
    weather_table = weather_tendency(store_nbr, year, month)

    weather = ['cool']

    item_nbr = units_table.index

    inner=dict()
    
    for feature in weather:
        for units in item_nbr:
            
            print(feature, units)
            
            a = weather_table[feature].copy()
            b = units_table.loc[units].copy()

            return a
            
            missing_index = (a.str.contains('M')) | (a.str.contains('T')) | (a.str.contains('-')) | (a.str.contains(' '))

            a[missing_index] = np.nan
            b[missing_index] = np.nan

            a.dropna(axis=0, inplace=True)
            b.dropna(axis=0, inplace=True)

            print('\t',a,b)
            
            
            inner[(units, feature)] = sp.stats.pearsonr(a.astype(float), b)
        
            correlation[(store_nbr, year,month)] = inner

    return correlation

In [10]:
len(weather_tendency(7,2012))

365